This is the overview of the peer graded assignment

In [70]:
#Import relevant libraries
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np

In [81]:
#import other relevant libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [71]:
#Scrape the website and store in Soup object
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

# Use pandas to convert into table
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [72]:
#Drop rows that have 'Not assigned' boroughs

indexNames = df[ df['Borough'] == 'Not assigned' ].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head(50)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [73]:
# Change value of Neighborhood to name of Borough if Neighborhood isnan

df.loc[df['Neighborhood'].isnull(), 'Borough'] = df['Neighborhood']
df.head(100)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
153,M1X,Scarborough,Upper Rouge
156,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
157,M5X,Downtown Toronto,"First Canadian Place, Underground city"
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"


In [74]:
# Group rows that have the same postal code number. 
UniquePC = df['Postal Code'].unique()

df = df.groupby([UniquePC]).agg(lambda col: ','.join(col))
df.tail(50)

,Postal Code,Borough,Neighborhood
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront"
M5B,M5B,Downtown Toronto,"Garden District, Ryerson"
M5C,M5C,Downtown Toronto,St. James Town
M5E,M5E,Downtown Toronto,Berczy Park
M5G,M5G,Downtown Toronto,Central Bay Street
M5H,M5H,Downtown Toronto,"Richmond, Adelaide, King"
M5J,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
M5K,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
M5L,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
M5M,M5M,North York,"Bedford Park, Lawrence Manor East"


In [75]:
df.shape

(103, 3)

In [76]:
#Import coordinates 
df_coordinates = pd.read_csv(r'C:\Users\Bilesh\Google Drive\Education\Google\Python Course\Coursera DataScience Capstone\CourseraCapstone\Geospatial_Coordinates.csv')

In [77]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [87]:
#Left join based on df using Postal Code as key
df_new = pd.merge(df,df_coordinates,on='Postal Code',how='left')
df_new.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [89]:
#Create subset of df for only Toronto
df_Toronto = df_new[df_new['Borough'].str.contains("Toronto")]
df_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [90]:
#Define user agent
address = 'Toronto'

geolocator = Nominatim(user_agent="TN_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [92]:
# Map of Toronto
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
        ).add_to(map_toronto)  
    
map_toronto


Observations: 
- The Boroughs appear to be clustered in the center of town and more spread out further away from town
- The airport is its own Borough
- The Boroughs mostly run along the coast